In [1]:
print('ok')

ok


In [2]:
%pwd

'/Users/ranyin/medical-chat-bot/research'

In [ ]:
import os
os.chdir("../")

In [5]:
%pwd

'/Users/ranyin/medical-chat-bot'

In [6]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# extract text from pdf files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob='**/*.pdf',
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_files('data')

In [9]:
extracted_data

[Document(metadata={'producer': '', 'creator': '', 'creationdate': '2022-04-09T14:22:21+00:00', 'moddate': '2022-04-12T14:05:56-05:00', 'title': "Harrison's Principles of Internal Medicine, Twenty-First Edition (Vol1 & Vol2)", 'keywords': '', 'author': 'Joseph Loscalzo; Anthony S. Fauci; Dennis L. Kasper; Stephen Hauser; Dan Longo; J. Larry Jameson', 'subject': '', 'source': "data/Harrison's Principles of Internal Medicine.pdf", 'total_pages': 15164, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': '', 'creator': '', 'creationdate': '2022-04-09T14:22:21+00:00', 'moddate': '2022-04-12T14:05:56-05:00', 'title': "Harrison's Principles of Internal Medicine, Twenty-First Edition (Vol1 & Vol2)", 'keywords': '', 'author': 'Joseph Loscalzo; Anthony S. Fauci; Dennis L. Kasper; Stephen Hauser; Dan Longo; J. Larry Jameson', 'subject': '', 'source': "data/Harrison's Principles of Internal Medicine.pdf", 'total_pages': 15164, 'page': 1, 'page_label': '2'}, page_conte

In [10]:
len(extracted_data)

15164

In [ ]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    '''
    filter the necessary data:
    only return source in metadata and the original page_content.
    '''
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get('source')
        minimal_docs.append(
            Document(
                page_content = doc.page_content,
                metadata = {'source':src}
            )
        )
    return minimal_docs


In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [13]:
minimal_docs

[Document(metadata={'source': "data/Harrison's Principles of Internal Medicine.pdf"}, page_content=''),
 Document(metadata={'source': "data/Harrison's Principles of Internal Medicine.pdf"}, page_content="Harrison's Principles of Internal\nMedicine, Twenty-First Edition (Vol1 &\nVol2)\n1. Cover\n2. Nav\n3. Cover\n4. Title Page\n5. Copyright Page\n6. Contents\n7. Contributors\n8. Preface\n9. Harrison’s Related Resources\n10. PART 1 The Profession of Medicine\n1. 1 The Practice of Medicine\n2. 2 Promoting Good Health\n3. 3 Vaccine Opposition and Hesitancy\n4. 4 Decision-Making in Clinical Medicine\n5. 5 Precision Medicine and Clinical Care\n6. 6 Screening and Prevention of Disease\n7. 7 Global Diversity of Health System Financing and Delivery\n8. 8 The Safety and Quality of Health Care\n9. 9 Diagnosis: Reducing Errors and Improving Quality\n10. 10 Racial and Ethnic Disparities in Health Care\n11. 11 Ethical Issues in Clinical Medicine\n12. 12 Palliative and End-of-Life Care\n11. PART 2 Ca

In [14]:
# seperate the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk



In [15]:
texts_chunk = text_split(minimal_docs)
print(f'Number of chunks: {len(texts_chunk)}')


Number of chunks: 60228


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    '''
    Download and return the HuggingFace embeddings model.
    '''
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    embeddings = HuggingFaceEmbeddings(
        model_name = model_name
    )
    return embeddings
embedding = download_embeddings()


/var/folders/m9/cg2vqmwx7vz12rsvvcd24p_h0000gn/T/ipykernel_16830/2444547677.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
vector = embedding.embed_query('Hello world')
vector

[-0.03447727859020233,
 0.031023221090435982,
 0.006734925322234631,
 0.026108991354703903,
 -0.03936200961470604,
 -0.16030248999595642,
 0.06692394614219666,
 -0.006441481411457062,
 -0.04745049774646759,
 0.014758859761059284,
 0.07087534666061401,
 0.055527571588754654,
 0.019193293526768684,
 -0.026251329109072685,
 -0.010109517723321915,
 -0.026940520852804184,
 0.02230745181441307,
 -0.02222665399312973,
 -0.14969265460968018,
 -0.01749308779835701,
 0.007676195353269577,
 0.05435231700539589,
 0.0032544690184295177,
 0.03172597661614418,
 -0.08462139964103699,
 -0.02940596640110016,
 0.05159565806388855,
 0.0481240451335907,
 -0.003314807079732418,
 -0.05827920883893967,
 0.04196929559111595,
 0.022210711613297462,
 0.1281888335943222,
 -0.022338921204209328,
 -0.011656290851533413,
 0.06292839348316193,
 -0.03287627175450325,
 -0.09122606366872787,
 -0.0311752837151289,
 0.052699554711580276,
 0.047034841030836105,
 -0.08420305699110031,
 -0.030056189745664597,
 -0.02074480988

In [19]:
print('Vector length:', len(vector))

Vector length: 384


In [20]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [21]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [22]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)



In [23]:
pc

In [27]:
from pinecone import ServerlessSpec

index_name = 'medical-chatbot'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric = 'cosine',
        spec = ServerlessSpec(cloud='aws', region='us-east-1')
    )
index = pc.Index(index_name)

In [28]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = texts_chunk,
    embedding = embedding,
    index_name = index_name
)

In [ ]:
# How to add more content to existing Pinecone Index
# newDoc = Document(
#     page_content = 'This is a new content',
#     metadata = {'source':'Youtube'}
# )
# docsearch.add_documents(documents=[newDoc])

In [32]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [33]:
retreived_docs = retriever.invoke('What is Acne?')
retreived_docs

[Document(id='bb1d5032-f21a-46e4-ac6b-e60de3ef444f', metadata={'source': "data/Harrison's Principles of Internal Medicine.pdf"}, page_content='Exogenous and endogenous factors can alter the expression of\nacne vulgaris. Friction and trauma (from headbands or chin straps of\nathletic helmets), application of comedogenic topical agents\n(cosmetics or hair preparations), or chronic topical exposure to\ncertain industrial compounds may elicit or aggravate acne.\nGlucocorticoids, topical or systemic, may also elicit acne. Other\nsystemic medications such as progestin-only contraception, lithium,'),
 Document(id='1f2c9689-4345-4ca3-8597-ab3160ad0dd9', metadata={'source': "data/Harrison's Principles of Internal Medicine.pdf"}, page_content='isoniazid, androgenic steroids, halogens, phenytoin, and\nphenobarbital may produce acneiform eruptions or aggravate\npreexisting acne. Genetic factors and polycystic ovary disease may\nalso play a role.\nTREATMENT\nAcne Vulgaris\nTreatment of acne vulgari

In [42]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model='gpt-5-nano')

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [44]:
system_prompt = (
    'You are a medical assistant for question-answering tasks.'
    'Use the following pieces of retrieved context to answer'
    'the question. If you do not know the answer, say that you'
    'do not know. Use three senteces maxium and keep the'
    'answer concise.'
    '\n\n'
    '{context}'
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)

In [45]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [47]:
response = rag_chain.invoke({'input':'what is heart attack?'})
print(response['answer'])

A heart attack, medically called a myocardial infarction, occurs when blood flow to part of the heart muscle is blocked, causing injury or death of that heart tissue. It often presents with ECG changes such as ST-segment elevation and elevated biomarkers of myocardial injury.
